In [1]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
# Read data
train = pd.read_csv('./data/train.csv')
labels = train.ix[:,0].values.astype('int32')
X_train = (train.ix[:,1:].values).astype('float32')
X_test = (pd.read_csv('./data/test.csv').values).astype('float32')

# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(labels) 

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
# pre-processing: divide by max and substract mean
scale = np.max(X_train)
X_train /= scale
X_test /= scale

mean = np.std(X_train)
X_train -= mean
X_test -= mean

In [4]:
input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

# Here's a Deep Dumb MLP (DDMLP)
model = Sequential()
model.add(Dense(128, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# we'll use categorical xent for the loss, and RMSprop as the optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [11]:
print("Training...")
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.1, verbose=2)

Training...
Train on 37800 samples, validate on 4200 samples
Epoch 1/10
23s - loss: 0.3687 - val_loss: 0.1800
Epoch 2/10
17s - loss: 0.2206 - val_loss: 0.1652
Epoch 3/10
17s - loss: 0.1902 - val_loss: 0.1768
Epoch 4/10
17s - loss: 0.1900 - val_loss: 0.1910
Epoch 5/10
17s - loss: 0.1926 - val_loss: 0.1842
Epoch 6/10
14s - loss: 0.1941 - val_loss: 0.2086
Epoch 7/10
15s - loss: 0.2013 - val_loss: 0.1919
Epoch 8/10
17s - loss: 0.1974 - val_loss: 0.2220
Epoch 9/10
16s - loss: 0.2065 - val_loss: 0.1999
Epoch 10/10
15s - loss: 0.2015 - val_loss: 0.1848


In [12]:
print("Generating test predictions...")
preds = model.predict_classes(X_test, verbose=0)

Generating test predictions...


In [13]:
def write_preds(preds, fname):
    pd.DataFrame({"ImageId": list(range(1,len(preds)+1)), "Label": preds}).to_csv(fname, index=False, header=True)

write_preds(preds, "keras-mlp.csv")